In [1]:
!wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz
!tar -xvzf tasks_1-20_v1-2.tar.gz

--2019-04-15 19:22:16--  http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz
Resolving www.thespermwhale.com (www.thespermwhale.com)... 69.65.3.213
Connecting to www.thespermwhale.com (www.thespermwhale.com)|69.65.3.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15719851 (15M) [application/x-gzip]
Saving to: ‘tasks_1-20_v1-2.tar.gz’

tasks_1-20_v1-2.tar 100%[===================>]  14.99M  9.70MB/s    in 1.5s    

2019-04-15 19:22:18 (9.70 MB/s) - ‘tasks_1-20_v1-2.tar.gz’ saved [15719851/15719851]

tasks_1-20_v1-2/
tasks_1-20_v1-2/hn/
tasks_1-20_v1-2/hn/qa16_basic-induction_train.txt
tasks_1-20_v1-2/hn/qa13_compound-coreference_train.txt
tasks_1-20_v1-2/hn/qa13_compound-coreference_test.txt
tasks_1-20_v1-2/hn/qa14_time-reasoning_test.txt
tasks_1-20_v1-2/hn/qa5_three-arg-relations_test.txt
tasks_1-20_v1-2/hn/qa17_positional-reasoning_train.txt
tasks_1-20_v1-2/hn/qa9_simple-negation_train.txt
tasks_1-20_v1-2/hn/qa12_conjunction_train.txt
tasks

In [13]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
keys = ['train', 'test']

In [15]:
def _parse(file):
    data, story = [], []
    for line in open(file).readlines():
        tid, text = line.rstrip('\n').split(' ', 1)
        if tid == '1':
            story = []
        # sentence
        if text.endswith('.'):
            story.append(text[:-1])
        # question
        else:
            # remove any leading or trailing whitespace after splitting
            query, answer, supporting = (x.strip() for x in text.split('\t'))
            substory = [x for x in story if x]
            data.append((substory, query[:-1], answer))    # remove '?'
            story.append("")
    return data

In [16]:
tasks = ['qa1_single-supporting-fact_', 'qa2_two-supporting-facts_', 'qa3_three-supporting-facts_']
data = {}
for t in tasks :
    data[t] = {}
    for k in keys :
        data[t][k] = list(zip(*_parse('tasks_1-20_v1-2/en-10k/' + t + k + '.txt')))

In [17]:
p, q, a = {}, {}, {}

for k in keys :
    p[k], q[k], a[k] = {}, {}, {}
    for t in tasks :
        p[k][t] = [" . ".join(x) for x in data[t][k][0]]
        q[k][t] = data[t][k][1]
        a[k][t] = data[t][k][2]

In [18]:
from Transparency.preprocess.vectorizer import Vectorizer
vec = Vectorizer(min_df=1)
flatten = lambda x : [y for z in x for y in z]
vec.fit(flatten(p['train'].values()) + flatten(q['train'].values()))

entity_list = [k for k, v in vec.word2idx.items() if v >= 4]
f = open('entity_list.txt', 'w')
f.write("\n".join(entity_list))
f.close()

In [21]:
import pandas as pd
for t in tasks :
    df_paragraphs = []
    df_questions = []
    df_answers = []
    df_exp_splits = []

    for k in keys :
        df_paragraphs += p[k][t]
        df_questions += q[k][t]
        df_answers += a[k][t]
        df_exp_splits += [k] * len(p[k][t])

    df = {'paragraph' : df_paragraphs, 'question' : df_questions, 'answer' : df_answers, 'exp_split' : df_exp_splits}
    df = pd.DataFrame(df)
    
    from sklearn.model_selection import train_test_split
    train_idx, dev_idx = train_test_split(df.index[df.exp_split == 'train'], test_size=0.15, random_state=16377)
    df.loc[dev_idx, 'exp_split'] = 'dev'
    df.to_csv('babi_' + t + 'dataset.csv', index=False)

In [8]:
for t in tasks :
    dataset = 'babi_' + t + 'dataset.csv'
    output_file = './vec_babi_' + t + '.p'
    %run "../preprocess_data_QA.py" --data_file {dataset} --output_file {output_file} --all_answers_file entity_list.txt \
    --word_vectors_type None --min_df 1

Vocabulary size :  24
Vocabulary size :  38
Vocabulary size :  39
